# RNN (LSTM)

## problem definition
- text classification

### import required packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

### load the data

In [2]:
from tensorflow.keras.datasets import imdb

# maxinum words per review = 10000
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=10000)

In [3]:
# get the embeddings
embeddings = imdb.get_word_index()

In [4]:
input_train.shape

(25000,)

In [5]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [6]:
embeddings.get('product')

2217

### pre-processing

In [13]:
from tensorflow.keras.preprocessing import sequence

# reshaping the input_train with 1000 values
input_train = sequence.pad_sequences(input_train, maxlen=1000)

In [14]:
input_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

### train the model

In [15]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

# create a model
model = Sequential()

# add embedding layer
# 10000: max length of the words
# 32: output length
model.add(Embedding(10000, 32))

# add simple RNN layer to connect all hidden layers to get output from previous layer
model.add(LSTM(32))

# add hidden layer
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(16))

# add the output layer
model.add(Dense(1, activation="sigmoid"))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# train the model
model.fit(input_train, y_train, epochs=1)

782/782 [==============================] - 92s 116ms/step - loss: 0.3799 - accuracy: 0.8244


### model evaluation

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize

# get the words and their embeddings
word_index = imdb.get_word_index()

def preprocess_review(review):
    # Tokenize the review
    tokens = word_tokenize(review.lower())
    sequences = [word_index.get(word, 0) for word in tokens]
    
    # Pad the sequence
    padded_sequence = pad_sequences([sequences], maxlen=1000)
    return padded_sequence

In [18]:
# new reviews to predict the sentiment
reviews = [
    "I really loved this movie! It was fantastic.",
    "This was the worst film I have ever seen.",
    "this product worst",
    "this a very bad product"
]


# convert all reviews to their embeddings
preprocessed_reviews = [preprocess_review(review) for review in reviews]

# predict each review using model
predictions = [model.predict(review) for review in preprocessed_reviews]
print(predictions)

for review, pred in zip(reviews, predictions):
    sentiment = 'Positive' if pred[0][0] > 0.5 else 'Negative'
    print(f'Review: "{review}" => Sentiment: {sentiment} (Probability: {pred[0][0]:.4f})')

1/1 [==============================] - 0s 16ms/step
[array([[0.7271924]], dtype=float32), array([[0.65820765]], dtype=float32), array([[0.5647126]], dtype=float32), array([[0.6298438]], dtype=float32)]
Review: "I really loved this movie! It was fantastic." => Sentiment: Positive (Probability: 0.7272)
Review: "This was the worst film I have ever seen." => Sentiment: Positive (Probability: 0.6582)
Review: "this product worst" => Sentiment: Positive (Probability: 0.5647)
Review: "this a very bad product" => Sentiment: Positive (Probability: 0.6298)
